In [7]:
import numpy as np
import pandas as pd

In [14]:
# Define a function for Markov chain simulation and calculate the first passage probability
# the mean occupation and recurrence time
def markovsimulation(states, ns, pt):
    # Initialize the time step count
    # Samples
    xlist = []
    ocp = []
    # Initialize the state
    xmm1 = 2
    # Simulate the transition process
    for j in range(ns):
        # Withdraw the corresponding transition probability
        i = xmm1 - 1
        pi = pt[i]
        # Sampling the state
        xm = np.random.choice(states, size=1, p=pi)[0]
        xlist.append(xm)
        xmm1 = xm
    # Initialize
    # To count the transitions
    count = {}
    f = {}
    # This part calculate the first passage probability
    for i in states:
        for j in states:
            count[(i,j)] = []
            # Find all the i,j tansitions and the corresponding steps
            m = 0
            # whether we found a i
            newi = 0
            newj = 0
            while m < len(xlist):
                if xlist[m] == i:
                    # If we find i then try to find j
                    h = m+1
                    while  h < len(xlist):
                        if xlist[h] == j:
                            # Count the steps needed
                            n = h - m
                            count[(i,j)].append(n)
                            # Stop search, make sure it is the first passage 
                            break
                        h += 1
                m +=1 
            # Evaluate the statistics
            statis = np.array(count[(i,j)])
            # Get all the possible n s 
            uniqn = np.unique(statis)
            fn = {}
            for n in uniqn:
                fn[n] = np.sum(statis==n)/len(statis)
            f[(i,j)] = fn
    # Calculate the probability qi,j(m)
    q = {}
    for i in states:
        for j in states:
            uniqn = np.unique(count[(i,j)])
            q[(i,j)] = {}
            fn = f[(i,j)]
            for mm in uniqn:
                q[(i,j)][mm] = 0
                # Calculate qi,j(m)
                for key, value in fn.items():
                    # Sum all the fi,j(n) while n<=m
                    if key <= mm:
                        q[(i,j)][mm] += value
                    else:
                        break
    # This part calculate the mean occupation time and recurrence time
    xlist = np.array(xlist)
    ocup = {}
    mocup = {}
    recur = {}
    mre = {}
    for i in states:
        ocup[i] = []
        recur[i] = []
        # Find the location of i
        indi = list(np.where(xlist == i)[0])
        # Calculate recurrence time
        for j in range(len(indi)-1):
            recur[i].append(indi[j+1]-indi[j])
        mre[i] = np.mean(np.array(recur[i]))
        # Find consecutive index groups
        new = 1
        k = 0
        while k < (len(indi)-1):
            # Check if it is a new search for goups of i
            if new == 1:
                length = 0
            # Check if the next term is consecutive
            if indi[k+1] == indi[k] + 1:
                length += 1
                new = 0
            else:
                ocup[i].append(length+1)
                # Start a new search
                new = 1
            k += 1
        mocup[i] = np.mean(np.array(ocup[i]))        
    
    return q, mocup, mre 

In [12]:
PT = np.array([[0.4, 0.5, 0.1], [0.3, 0.3, 0.4], [0.1, 0.7, 0.2]])
NS = 3000000
States = [1, 2, 3]

In [15]:
Q, MOCUP, MRE = markovsimulation(States, NS, PT)

In [19]:
# Output qi,j(m), ms is the matrix size
def outputq(q, m, ms):
    qmatrix = np.zeros((ms, ms))
    transitions = list(q.keys())
    for transit in transitions:
        (i,j) = transit
        qmatrix[i-1][j-1] = q[(i,j)][m]
    return qmatrix     

In [17]:
MS = 3

In [34]:
M = 19
Qmatrix = outputq(Q, M, MS)

In [35]:
Qmatrix

array([[0.99318998, 1.        , 0.99614071],
       [0.99162936, 0.99999855, 0.99732336],
       [0.99000876, 1.        , 0.99694027]])

In [36]:
MOCUP

{1: 1.665827847578062, 2: 1.4282924510756132, 3: 1.251567023285085}

In [7]:
# Test the code using two state system
TPT = np.array([[0.8, 0.2], [0.5, 0.5]])
TNS = 100000
TStates = [1, 2]

In [8]:
TQ, TMOCUP, TMRE = markovsimulation(TStates, TNS, TPT)

[1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 2, 2, 2, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2, 1, 1, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 2, 2, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 2, 1, 1, 

{(1, 1): [1, 1, 1, 1, 1, 4, 1, 1, 5, 1, 1, 1, 1, 1, 3, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 3, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 3, 1, 1, 5, 3, 1, 5, 1, 3, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 5, 1, 1, 1, 1, 1, 3, 1, 2, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 3, 5, 1, 2, 2, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 3, 1, 2, 1, 1, 4, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 1, 7, 2, 1, 3, 3, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 1, 1, 1, 1, 1, 1, 1, 1, 

In [43]:
# Simulate a Markov chain with absorbing state and calculate mean time to arrive at absorbing
# i is the initial state, r is the absorbing state
def markovabssimulation(states, ns, pt, j, r):
    # Initialize the time step count
    # Samples
    mj = []
    # Simulate the transition process
    for k in range(ns):
        mjk = 0
        xmlist = []
        # Initialize the state to be i
        xmm1 = j
        while True:
            # Withdraw the corresponding transition probability
            i = xmm1 - 1
            pi = pt[i]
            # Sampling the state
            xm = np.random.choice(states, size=1, p=pi)[0]
            xmlist.append(xm)
            if xm == r:
                # Count the last step 
                mjk += 1
                mj.append(mjk)
                break
            else:
                # Continue sampling
                xmm1 = xm
                mjk += 1
        
    mmj = np.mean(mj)
    return mmj

In [39]:
PT = np.array([[0.4, 0.5, 0.1], [0.3, 0.3, 0.4], [0, 0, 1]])
States = [1, 2, 3]
R = 3

In [57]:
NS = 3000000
J = 2

In [58]:
MMJ = markovabssimulation(States, NS, PT, J, R)

In [59]:
MMJ

3.3344183333333333

In [60]:
print(12/2.7)

4.444444444444444


In [19]:
PT = np.array([[0.4, 0.5, 0.1], [0.3, 0.3, 0.4], [0.1, 0.7, 0.2]])
a = np.array([0.274, 0.461, 0.265])
print(np.matmul(a, PT))

[0.2744 0.4608 0.2648]


In [3]:
print((0.5*0.274+0.3*0.461)/0.735)

0.3745578231292517


In [5]:
print(0.375+0.337)

0.712


In [8]:
PT = np.array([[0.712, 0.288], [0.8, 0.2]])
a = np.array([0.735, 0.265])
print(np.matmul(a, PT))

[0.73532 0.26468]


In [9]:
print(0.8/1.088)

0.7352941176470588
